In [1]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

/home/ai02/miniconda3/envs/kk/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ai02/miniconda3/envs/kk/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/ai02/miniconda3/envs/kk/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [15]:
model = CLIPModel.from_pretrained('/home/ai02/Desktop/clip-vit-base-patch32')
processor = CLIPProcessor.from_pretrained('/home/ai02/Desktop/clip-vit-base-patch32')
image = Image.open('/media/ai02/43633275BF4935F0/kk/datasets/RUOD/images/test/003926.jpg')

In [16]:
text = ['God', 'turtle', 'bus']
inputs = processor(text=text, images=image, return_tensors='pt', padding=True)

output = model(**inputs)
logits = output.logits_per_image

In [17]:
print(logits.softmax(dim=1))

tensor([[2.1311e-04, 9.9976e-01, 2.5429e-05]], grad_fn=<SoftmaxBackward0>)
